<a href="https://www.kaggle.com/code/iwways/insurance-premiums-unlocked-train-like-a-pro?scriptVersionId=213838395" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

Hi!
This is new playground competition and in this month we're having an insurance dataset. 

Before diving deeper into code, let's describe a little topic in which we will work.

For start, a little meme which describes all this thing :)

![](https://leadsurance.com/wp-content/uploads/2020/08/insurance-meme-6.jpg)


In my previous notebook I was combined EDA and training
https://www.kaggle.com/code/iwways/calculating-premiums-like-a-pro-or-not

But I decided to make training in separate notebook, cause a lot of things is going here :)

# Import libraries

In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_log_error
import pandas as pd
import missingno as mnso
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import boxcox
from scipy.special import boxcox1p


# Load data

In [2]:
X_train = pd.read_csv('/kaggle/input/playground-series-s4e12/train.csv')
X_test = pd.read_csv('/kaggle/input/playground-series-s4e12/test.csv')
sample_submission = pd.read_csv('/kaggle/input/playground-series-s4e12/sample_submission.csv')

X_train = X_train.drop(['id'], axis=1)
id_test = X_test.id
X_test = X_test.drop('id', axis=1)
# y = X_train['Premium Amount']
# y_log = np.log1p(y)

pd.set_option('display.max_columns', None)


After exploring discussions it seems reasonable to log transform our target variable to ensure that we optimizing directly the evaluation metric for this competition

# Feature enginereering

It's will be useful if we separate Policy Start Date feature to different features, as day, month and other.

## Time features

In [3]:
X_train

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Policy Start Date,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,2023-12-23 15:21:39.134960,Poor,No,Weekly,House,2869.0
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,2023-06-12 15:21:39.111551,Average,Yes,Monthly,House,1483.0
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,2023-09-30 15:21:39.221386,Good,Yes,Weekly,House,567.0
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,2024-06-12 15:21:39.226954,Poor,Yes,Daily,Apartment,765.0
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,2021-12-01 15:21:39.252145,Poor,Yes,Weekly,House,2022.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,NaN,5.0,372.0,3.0,2023-05-03 15:21:39.257696,Poor,No,Daily,Apartment,1303.0
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,NaN,10.0,597.0,4.0,2022-09-10 15:21:39.134960,Poor,No,Weekly,Apartment,821.0
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,0.0,19.0,NaN,6.0,2021-05-25 15:21:39.106582,Good,No,Monthly,Condo,371.0
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,1.0,7.0,407.0,4.0,2021-09-19 15:21:39.190215,Poor,No,Daily,Apartment,596.0


In [4]:
# def create_policy_start_date_separated_features(df):
#     df['Policy Start Date'] = pd.to_datetime(df['Policy Start Date'], errors='coerce')
#     df['Policy_Start_Year'] = df['Policy Start Date'].dt.year
#     df['Policy_Start_Month'] = df['Policy Start Date'].dt.month
#     df['Policy_Start_Day'] = df['Policy Start Date'].dt.day
    

#     return df


In [5]:
import numpy as np
import pandas as pd

def circular_features_time(df, date_column='Policy Start Date'):
    df[date_column] = pd.to_datetime(df[date_column])
    
    df['day_of_week'] = df[date_column].dt.dayofweek
    df['day_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
    df['day_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)
    
    df['day_of_year'] = df[date_column].dt.dayofyear
    df['year_day_sin'] = np.sin(2 * np.pi * df['day_of_year'] / 365)
    df['year_day_cos'] = np.cos(2 * np.pi * df['day_of_year'] / 365)
    
    df['month'] = df[date_column].dt.month
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)
    
    df["seconds_since_1970"] = df[date_column].astype("int64") // 10**9
    df = df.drop('Policy Start Date', axis=1)

    return df

    

In [6]:
# train = create_policy_start_date_separated_features(X_train)
# test = create_policy_start_date_separated_features(X_test)

train = circular_features_time(X_train)
test = circular_features_time(X_test)

In [7]:
train

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,Premium Amount,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970
0,19.0,Female,10049.0,Married,1.0,Bachelor's,Self-Employed,22.598761,Urban,Premium,2.0,17.0,372.0,5.0,Poor,No,Weekly,House,2869.0,5,-0.974928,-0.222521,357,-0.137279,0.990532,12,-2.449294e-16,1.000000e+00,1703344899
1,39.0,Female,31678.0,Divorced,3.0,Master's,NaN,15.569731,Rural,Comprehensive,1.0,12.0,694.0,2.0,Average,Yes,Monthly,House,1483.0,0,0.000000,1.000000,163,0.329408,-0.944188,6,1.224647e-16,-1.000000e+00,1686583299
2,23.0,Male,25602.0,Divorced,3.0,High School,Self-Employed,47.177549,Suburban,Premium,1.0,14.0,NaN,3.0,Good,Yes,Weekly,House,567.0,5,-0.974928,-0.222521,273,-0.999917,-0.012910,9,-1.000000e+00,-1.836970e-16,1696087299
3,21.0,Male,141855.0,Married,2.0,Bachelor's,NaN,10.938144,Rural,Basic,1.0,0.0,367.0,1.0,Poor,Yes,Daily,Apartment,765.0,2,0.974928,-0.222521,164,0.313107,-0.949718,6,1.224647e-16,-1.000000e+00,1718205699
4,21.0,Male,39651.0,Single,1.0,Bachelor's,Self-Employed,20.376094,Rural,Premium,0.0,8.0,598.0,4.0,Poor,Yes,Weekly,House,2022.0,2,0.974928,-0.222521,335,-0.493776,0.869589,12,-2.449294e-16,1.000000e+00,1638372099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,Female,27316.0,Married,0.0,Master's,Unemployed,13.772907,Urban,Premium,NaN,5.0,372.0,3.0,Poor,No,Daily,Apartment,1303.0,2,0.974928,-0.222521,123,0.854322,-0.519744,5,5.000000e-01,-8.660254e-01,1683127299
1199996,54.0,Male,35786.0,Divorced,NaN,Master's,Self-Employed,11.483482,Rural,Comprehensive,NaN,10.0,597.0,4.0,Poor,No,Weekly,Apartment,821.0,5,-0.974928,-0.222521,253,-0.936881,-0.349647,9,-1.000000e+00,-1.836970e-16,1662823299
1199997,19.0,Male,51884.0,Divorced,0.0,Master's,NaN,14.724469,Suburban,Basic,0.0,19.0,NaN,6.0,Good,No,Monthly,Condo,371.0,1,0.781831,0.623490,145,0.601624,-0.798779,5,5.000000e-01,-8.660254e-01,1621956099
1199998,55.0,Male,NaN,Single,1.0,PhD,NaN,18.547381,Suburban,Premium,1.0,7.0,407.0,4.0,Poor,No,Daily,Apartment,596.0,6,-0.781831,0.623490,262,-0.979614,-0.200891,9,-1.000000e+00,-1.836970e-16,1632064899


In [8]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200000 entries, 0 to 1199999
Data columns (total 29 columns):
 #   Column                Non-Null Count    Dtype  
---  ------                --------------    -----  
 0   Age                   1181295 non-null  float64
 1   Gender                1200000 non-null  object 
 2   Annual Income         1155051 non-null  float64
 3   Marital Status        1181471 non-null  object 
 4   Number of Dependents  1090328 non-null  float64
 5   Education Level       1200000 non-null  object 
 6   Occupation            841925 non-null   object 
 7   Health Score          1125924 non-null  float64
 8   Location              1200000 non-null  object 
 9   Policy Type           1200000 non-null  object 
 10  Previous Claims       835971 non-null   float64
 11  Vehicle Age           1199994 non-null  float64
 12  Credit Score          1062118 non-null  float64
 13  Insurance Duration    1199999 non-null  float64
 14  Customer Feedback     1122176 non-

## Handling missing values TODO

After Chris's Deotte psot i decided not to impute values for NaN's, so let's check how our results will be.

In [9]:
def impute_missing_numerical_data(df):
    num = df.select_dtypes(exclude='object').columns
    for n in num:
        df[n] = df[n].fillna(-1)

    return df

def impute_missing_categorical_data(df):
    cat = df.select_dtypes(include='object').columns
    for c in cat:
        df[c] = df[c].fillna('Unknown')

    return df

In [10]:
#train = impute_missing_categorical_data(train)
#test = impute_missing_categorical_data(test)

In [11]:
#train = impute_missing_numerical_data(train)
#test = impute_missing_numerical_data(test)

In [12]:
#train.info()

## Encoding categorical features

Now it's time for encoding!

In [13]:
def encode_ordinal(df):
    educ = {"High School":0, "Bachelor's":1, "Master's":2, "PhD":3}
    policy = {'Basic':0, 'Comprehensive':1, 'Premium':2}
    exerc = {'Rarely':0, 'Daily':1, 'Weekly':2, 'Monthly': 3}
    feedback = {'Poor':0, 'Average':1, 'Good':2, "Unknown": 0}

    df['Education Level'] = df['Education Level'].map(educ)
    df['Policy Type'] = df['Policy Type'].map(policy)
    df['Exercise Frequency'] = df['Exercise Frequency'].map(exerc)
    df['Customer Feedback'] = df['Customer Feedback'].map(feedback)
    return df

In [14]:
def encode_binary(df):
    df['Gender'] = df['Gender'].map({'Male':0, 'Female':1})
    df['Smoking Status'] = df['Smoking Status'].map({'Yes':1, 'No':0})
    return df

In [15]:
def one_hot_dummies(df, categorical):
    oh = pd.get_dummies(df[categorical])
    df = df.drop(categorical, axis=1)
    return pd.concat([df, oh], axis=1)

In [16]:
y = X_train['Premium Amount']
train = train.drop('Premium Amount', axis=1)
y_log = np.log1p(y)

In [17]:
train = encode_binary(train)
test = encode_binary(test)

In [18]:
train = encode_ordinal(train)
test = encode_ordinal(test)

In [19]:
train

,Age,Gender,Annual Income,Marital Status,Number of Dependents,Education Level,Occupation,Health Score,Location,Policy Type,Previous Claims,Vehicle Age,Credit Score,Insurance Duration,Customer Feedback,Smoking Status,Exercise Frequency,Property Type,day_of_week,day_sin,day_cos,day_of_year,year_day_sin,year_day_cos,month,month_sin,month_cos,seconds_since_1970
0,19.0,1,10049.0,Married,1.0,1,Self-Employed,22.598761,Urban,2,2.0,17.0,372.0,5.0,0.0,0,2,House,5,-0.974928,-0.222521,357,-0.137279,0.990532,12,-2.449294e-16,1.000000e+00,1703344899
1,39.0,1,31678.0,Divorced,3.0,2,NaN,15.569731,Rural,1,1.0,12.0,694.0,2.0,1.0,1,3,House,0,0.000000,1.000000,163,0.329408,-0.944188,6,1.224647e-16,-1.000000e+00,1686583299
2,23.0,0,25602.0,Divorced,3.0,0,Self-Employed,47.177549,Suburban,2,1.0,14.0,NaN,3.0,2.0,1,2,House,5,-0.974928,-0.222521,273,-0.999917,-0.012910,9,-1.000000e+00,-1.836970e-16,1696087299
3,21.0,0,141855.0,Married,2.0,1,NaN,10.938144,Rural,0,1.0,0.0,367.0,1.0,0.0,1,1,Apartment,2,0.974928,-0.222521,164,0.313107,-0.949718,6,1.224647e-16,-1.000000e+00,1718205699
4,21.0,0,39651.0,Single,1.0,1,Self-Employed,20.376094,Rural,2,0.0,8.0,598.0,4.0,0.0,1,2,House,2,0.974928,-0.222521,335,-0.493776,0.869589,12,-2.449294e-16,1.000000e+00,1638372099
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1199995,36.0,1,27316.0,Married,0.0,2,Unemployed,13.772907,Urban,2,NaN,5.0,372.0,3.0,0.0,0,1,Apartment,2,0.974928,-0.222521,123,0.854322,-0.519744,5,5.000000e-01,-8.660254e-01,1683127299
1199996,54.0,0,35786.0,Divorced,NaN,2,Self-Employed,11.483482,Rural,1,NaN,10.0,597.0,4.0,0.0,0,2,Apartment,5,-0.974928,-0.222521,253,-0.936881,-0.349647,9,-1.000000e+00,-1.836970e-16,1662823299
1199997,19.0,0,51884.0,Divorced,0.0,2,NaN,14.724469,Suburban,0,0.0,19.0,NaN,6.0,2.0,0,3,Condo,1,0.781831,0.623490,145,0.601624,-0.798779,5,5.000000e-01,-8.660254e-01,1621956099
1199998,55.0,0,NaN,Single,1.0,3,NaN,18.547381,Suburban,2,1.0,7.0,407.0,4.0,0.0,0,1,Apartment,6,-0.781831,0.623490,262,-0.979614,-0.200891,9,-1.000000e+00,-1.836970e-16,1632064899


In [20]:
numerical_features_total = test.select_dtypes(exclude='object').columns
numerical_features_continuos = test.select_dtypes(exclude=['object', 'int']).columns
numerical_features_discrete = test.select_dtypes(exclude=['object', 'float']).columns
categorical_features = test.select_dtypes(include='object').columns

In [21]:
categorical_features

Index(['Marital Status', 'Occupation', 'Location', 'Property Type'], dtype='object')

In [22]:
train_enc = one_hot_dummies(train, categorical_features)
test_enc = one_hot_dummies(test, categorical_features)

## Feature selection

Saving some samples for latter to evaluate perfomance

In [23]:
# train_enc, X_test_separated, y_log, y_test_separated = train_test_split(train_enc, y_log, test_size=0.15, random_state=42)

# Training model

In [24]:
def rmsle(y_true, y_pred):
    
    log_true = np.log1p(y_true)  # log(1 + y_true)
    log_pred = np.log1p(y_pred)  # log(1 + y_pred)

    return np.sqrt(mean_squared_error(log_true, log_pred))


In [25]:
x_train, x_val, y_train, y_val = train_test_split(train_enc, y_log, test_size=0.15, random_state=42)

## XGBoost Regressor

In [26]:
import xgboost as xgb

xgb_model = xgb.XGBRegressor(learning_rate=0.2, objective = 'reg:squarederror', n_estimators=300, gamma=0.5, max_depth=20, max_leaves=5, random_state=42)
xgb_model.fit(x_train, y_train, eval_set=[(x_val, y_val)])

[0]	validation_0-rmse:1.08933
[1]	validation_0-rmse:1.08432
[2]	validation_0-rmse:1.08106
[3]	validation_0-rmse:1.07863
[4]	validation_0-rmse:1.07672
[5]	validation_0-rmse:1.07522
[6]	validation_0-rmse:1.07377
[7]	validation_0-rmse:1.07267
[8]	validation_0-rmse:1.07110
[9]	validation_0-rmse:1.07017
[10]	validation_0-rmse:1.06948
[11]	validation_0-rmse:1.06869
[12]	validation_0-rmse:1.06794
[13]	validation_0-rmse:1.06741
[14]	validation_0-rmse:1.06686
[15]	validation_0-rmse:1.06635
[16]	validation_0-rmse:1.06590
[17]	validation_0-rmse:1.06544
[18]	validation_0-rmse:1.06504
[19]	validation_0-rmse:1.06460
[20]	validation_0-rmse:1.06428
[21]	validation_0-rmse:1.06395
[22]	validation_0-rmse:1.06364
[23]	validation_0-rmse:1.06324
[24]	validation_0-rmse:1.06301
[25]	validation_0-rmse:1.06269
[26]	validation_0-rmse:1.06243
[27]	validation_0-rmse:1.06222
[28]	validation_0-rmse:1.06203
[29]	validation_0-rmse:1.06183
[30]	validation_0-rmse:1.06166
[31]	validation_0-rmse:1.06151
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0.5, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.2, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=20, max_leaves=5,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=300, n_jobs=None,
             num_parallel_tree=None, random_state=42, ...)

In [27]:
pred_xgb = np.expm1(xgb_model.predict(x_val))
y_val_true = np.expm1(y_val)
train_min = y_train.min()
clipped_predictions = np.maximum(pred_xgb, train_min)

print(rmsle(y_val_true, pred_xgb)) 



1.05494211781101


## LightGBM Regressor

In [28]:
import lightgbm as light

light_model = light.LGBMRegressor(learning_rate = 0.2, max_depth=10, num_leaves=2,n_estimators = 300, random_state=42, verbose=0)
light_model.fit(x_train, y_train, eval_set=[(x_val, y_val)])

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


LGBMRegressor(learning_rate=0.2, max_depth=10, n_estimators=300, num_leaves=2,
              random_state=42, verbose=0)

In [29]:
pred_light = np.expm1(light_model.predict(x_val))
y_val_true = np.expm1(y_val)
train_min = y_train.min()
clipped_predictions = np.maximum(pred_light, train_min)

print(rmsle(y_val_true, pred_light)) 


1.0714758233186188


## CatBoost

In [30]:
from catboost import CatBoostRegressor
cat_model = CatBoostRegressor(iterations=1000,
                          early_stopping_rounds=100,
                          grow_policy = 'Depthwise',
                          depth=8,
                          random_state=42,
                          l2_leaf_reg = 1,
                          learning_rate=0.03,
                             verbose=0)
cat_model.fit(train_enc, y_log)

In [31]:
pred = np.expm1(cat_model.predict(x_val))
y_val_true = np.expm1(y_val)
train_min = y_train.min()
clipped_predictions = np.maximum(pred, train_min)

print(rmsle(y_val_true, pred)) 

1.0199254472041088


In [32]:
# from catboost import CatBoostRegressor
# from sklearn.model_selection import GridSearchCV

# param_grid = {
#     'iterations': [2000, 5000],
#     'depth': [8, 10, 12],
#     'learning_rate': [0.01, 0.03],
#     'l2_leaf_reg': [1, 3, 5],
#     'verbose': [0]
# }

# model = CatBoostRegressor()
# grid = GridSearchCV(model, param_grid, cv=3)
# grid.fit(train_enc, y_log)

# print(grid.best_params_)


# Bias-Variance Tradeoff

Recently I started diving into bias-variance tradeoff, and one of the suggestions were to plot graph of residuals between model's predicted and true values. So, let's take a look

In [33]:
# pred = cat_model.predict(X_test_separated)
# # y_test_separated = (y_test_separated)
# residuals = y_test_separated-pred
# print(residuals)
# plt.scatter(y_test_separated, residuals)
# plt.axhline(y=0, color='red', linestyle='--')  # Лінія нульових залишків

# plt.show()

In [34]:
# sns.kdeplot(x=pred)

In [35]:
# sns.kdeplot(x=y_test_separated)

In [36]:
# from sklearn.model_selection import learning_curve

# train_sizes, train_scores, test_scores = learning_curve(cat_model, train_enc, y_log, cv=5)

# train_mean = np.mean(train_scores, axis=1)
# test_mean = np.mean(test_scores, axis=1)
# train_std = np.std(train_scores, axis=1)
# test_std = np.std(test_scores, axis=1)

# plt.plot(train_sizes, train_mean, label="Training score", color="blue")
# plt.plot(train_sizes, test_mean, label="Cross-validation score", color="green")
# plt.fill_between(train_sizes, train_mean - train_std, train_mean + train_std, color="blue", alpha=0.2)
# plt.fill_between(train_sizes, test_mean - test_std, test_mean + test_std, color="green", alpha=0.2)

# plt.title("Learning Curve (Bias-Variance Tradeoff)")
# plt.xlabel("Training Size")
# plt.ylabel("Score")
# plt.legend(loc="best")
# plt.grid(True)
# plt.show()

# CV score

After my posted discussion I decided not to follow public leaderboard, but instead follow cross validation and kfold score for models

In [37]:
from sklearn.metrics import make_scorer

def rmsle_on_original_scale(y_true, y_pred):
    y_true = np.expm1(y_true)
    y_pred = np.expm1(y_pred)
    
    y_pred = np.maximum(y_pred, np.min(y_true))
    
    log_true = np.log1p(y_true)
    log_pred = np.log1p(y_pred)
    return np.sqrt(np.mean((log_true - log_pred) ** 2))

rmsle_scorer = make_scorer(rmsle_on_original_scale, greater_is_better=False)

In [38]:
from sklearn.model_selection import cross_val_score, StratifiedKFold
def perform_cv_score(estimator, X, y, scoring):
    
    scores = cross_val_score(estimator, X=X, 
                             y=y,scoring=scoring, cv = 10, 
                             n_jobs=-1,  error_score="raise")

    return scores

In [39]:
data = {
        "XGBRegressor": np.average(-1*perform_cv_score(xgb_model, train_enc, y_log, rmsle_scorer)),       
       "LightGBM": np.average(-1*perform_cv_score(light_model, train_enc, y_log, rmsle_scorer)),
        "CatBoost": np.average(-1*perform_cv_score(cat_model, train_enc, y_log, rmsle_scorer))
       }

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines


In [40]:
data

{'XGBRegressor': 1.0532725394040727,
 'LightGBM': 1.0704004591013414,
 'CatBoost': 1.0470933153492425}

# Submission

In [41]:
predicted = xgb_model.predict(test_enc)

submission = pd.DataFrame({'id': id_test, 'Premium Amount': np.expm1(predicted)})
submission.to_csv("submission_insurance_xgb.csv", index=False)

In [42]:
predicted = light_model.predict(test_enc)

submission = pd.DataFrame({'id': id_test, 'Premium Amount': np.expm1(predicted)})
submission.to_csv("submission_insurance_light.csv", index=False)

In [43]:
predicted = cat_model.predict(test_enc)

submission = pd.DataFrame({'id': id_test, 'Premium Amount': np.expm1(predicted)})
submission.to_csv("submission_insurance_cat.csv", index=False)

In [44]:
predicted_сat = np.expm1(cat_model.predict(test_enc))
predicted_xgb = np.expm1(xgb_model.predict(test_enc))
predicted_light = np.expm1(light_model.predict(test_enc))

submission = pd.DataFrame({'id': id_test, 'Premium Amount': (predicted_сat+predicted_light+predicted_xgb)/3})
submission.to_csv("submission_insurance_ensemble.csv", index=False)